# Reservoir operation under uncertainty
In this section by means of an interactive example putting you into the role of a water system manager we will introduce the concept of decision making under uncertainty and different approaches to deal with forecast uncertainty.

<left><img src="Images/Dam.gif" width = "400px"><left>
    
## Decision making under uncertainty
Imagine that as a manager you receive every week both a hydroclimate and a demand forecast for the next 8 weeks and must define a pumped inflow scheduling for this period. Each forecast contains 10 members or possible scenarios with the same probability of occurrence. Here we present 2 different approaches to address the problem of defining a policy under uncertainty.
    
Main sources of uncertainty for this example:
    
**1) Hydroclimate forecast**

<right> <img src="Images/Hydroclimatic_forecast3.gif" width = "400px"><right>

**2) Demand forecast**
    
<right> <img src="Images/Demand1.gif" width = "400px"><right>
    
### Approach 1: deterministic scenario
This approach considers a single future scenario. For instance, a conservative and traditioanlly applied approach consists of assuming the worst-case scenario from the different possible combinations of inflow and demand forecast, i.e. we choose the lowest inflow and highest demand scenario. A less conservative but more problable approach is to consider the average of the 10 forecast members as the future scenario. Among the 10 forecast members select manually any scenario depending on your criteria.

**First of all, we need to import the necessary libraries to run the model:**

In [1]:
from bqplot import pyplot as plt
from bqplot import *
from bqplot.traits import *
import numpy as np
import ipywidgets as widgets
from IPython.display import display
from platypus import NSGAII, Problem, Real, Integer

from Subroutines.clim_dem_forecast import forecast
from Subroutines.Forecast_ensemble import Ensemble_member_sel, Observed_inflows, Forecast_ensemble
from Subroutines.Interactive_pump_schedule import Interactive_Pareto_front_det, Interactive_Pareto_front_act, Interactive_Pareto_front

#### Inputs definition
**1. Simulation time:** we introduce the number of weeks ($simtime$) that we would like to simulate. In this case 8 weeks.

In [2]:
simtime = 8 # weeks

**2. Inputs:** forecast of weekly natural inflows ($I$), evaporation ($E$), temperature ($T$) and water demand ($d$). They will be automatically generated by a simplistic forecast model. This model generates an array for each forecast member of random but constrained values.

In [3]:
members_num = 10 # 10 forecast members
I_for,T_for,E_for,d_for,uncertain = forecast(simtime,members_num) # in ML/day

**Plot and selection of the forecast members**

Amongh the 10 forecast members select a inflow forecast member and a demand forecast member according to your criteria.

In [4]:
fig_1a,fig_1b,I_sel,T_sel,E_sel,d_sel = Ensemble_member_sel(simtime,members_num,I_for,T_for,E_for,d_for)
widgets.VBox([fig_1a,fig_1b])

**3. Initial conditions:** initial reservoir storage volume.

In [5]:
### Initial conditions ###
  # Initial storage volume
S0 = 40 # in ML

**4. System constraints:** Now we define the constraints of the system such as the reservoir storage capacity or the pumping energy cost.

In [6]:
### Constraints ###
  # Max storage volume
Smax = 150 #  in ML
  # Min storage volume
Smin = 0 # in ML
  # Min environmental compensation flow
env_min = 2 # in ML/week

# Pumping energy cost per ML
c = 50 # £/ML

#### 2) Definition of the water resoirvoir system model
    
<left> <img src="Images/system_representation_IO2.3.png" width = "600px"><left>

In [7]:
from Subroutines.Water_system_model import Water_system_model as syst_sim

#### 3) Decision making: pumping policy
**Optimization**

In [8]:
# Optimizer
def auto_optim(vars):
    
    pinflow1 = vars[0]
    pinflow2 = vars[1]
    pinflow3 = vars[2]
    pinflow4 = vars[3]
    pinflow5 = 0
    pinflow6 = 0
    pinflow7 = 0
    pinflow8 = 0
    
    u = np.array([pinflow1,pinflow2,pinflow3,pinflow4,pinflow5,pinflow6,pinflow7,pinflow8])
    S,env,w,r = syst_sim(simtime,I_sel+u,E_sel,d_sel,S0,Smax,env_min)
    
    sdpen = (np.sum((np.maximum(d_sel-r,[0]*simtime))**2)).astype('int')
    pcost = (np.sum(np.array(u)*c)).astype('int')
    
    return [sdpen,pcost]

problem = Problem(4,2)
Real0 = Real(0, 40); Real1 = Real(0, 40); Real2 = Real(0, 40); Real3 = Real(0, 40)

problem.types[:] = [Real0] + [Real1] + [Real2] + [Real3]
problem.function = auto_optim

population_size = 20
algorithm = NSGAII(problem,population_size)
algorithm.run(10000) # Number of iterations

results1_optim_relea_2 = np.array([algorithm.result[i].objectives[0] for i in range(population_size)])
results2_optim_relea_2 = np.array([algorithm.result[i].objectives[1] for i in range(population_size)])

solutions_optim_relea_2 = [[algorithm.result[i].variables[0],algorithm.result[i].variables[1],algorithm.result[i].variables[2],
                            algorithm.result[i].variables[3],0,0,0,0] for i in range(population_size)]

**Pareto front and selection of the policy**

In [9]:
fig_2pf,fig_2b,fig_2c,fig_2d,pareto_det = Interactive_Pareto_front_det(simtime,I_sel,E_sel,d_sel,S0,Smax,Smin,env_min,c, 
                        solutions_optim_relea_2,results1_optim_relea_2,results2_optim_relea_2)
widgets.VBox([widgets.HBox([widgets.VBox([fig_2d,fig_2b]),fig_2pf]),widgets.HBox([fig_2c])])

#### 4) Eight weeks later: evaluation of the system performance
After 8 weeks it is time to evaluate the system performance against the actual inflows.

<img src="Images/Calendar.png" width="400px"/>


**Observed inflows in the last 8 weeks**


In [10]:
I_act,T_act,E_act,d_act,fig_3a,fig_3b = Observed_inflows(simtime,members_num,I_sel,d_sel,I_for,d_for)
widgets.VBox([fig_3a,fig_3b])

**Performance of the system after applying the selected policy**

In [11]:
fig_4b,fig_4c,fig_4d,fig_4pf = Interactive_Pareto_front_act(simtime,I_act,E_act,d_act,S0,Smax,Smin,env_min,c,
                                            solutions_optim_relea_2,results1_optim_relea_2,results2_optim_relea_2,pareto_det.selected[0])
widgets.VBox([widgets.HBox([widgets.VBox([fig_4d,fig_4b]),fig_4pf]),widgets.HBox([fig_4c])])

### Approach 2: treating the forecast as an ensemble
This approach considers and simulates all the members of the forecast.
#### 1) Forecast ensemble

**Plot of the forecast members**

In [12]:
fig_9a,fig_9b = Forecast_ensemble(simtime,members_num,I_for,d_for)
widgets.VBox([fig_9a,fig_9b])

#### 2) Decision making: release policy
**Optimization**

In [13]:
from platypus import NSGAII, Problem, Real, Integer

def auto_optim_v2(vars):
    
    pinflow1 = vars[0]
    pinflow2 = vars[1]
    pinflow3 = vars[2]
    pinflow4 = vars[3]
    pinflow5 = 0
    pinflow6 = 0
    pinflow7 = 0
    pinflow8 = 0
    
    u = np.array([pinflow1,pinflow2,pinflow3,pinflow4,pinflow5,pinflow6,pinflow7,pinflow8])
    S,env,w,r = syst_sim(simtime,I_for+u,E_for,d_for,S0,Smax,env_min)
    
    sdpen_mean = np.mean(np.sum(np.maximum(d_for-r,np.zeros(np.shape(d_for)))**2,axis=1))
    pcost = np.sum(np.array(u)*c)
    
    return [sdpen_mean,pcost]

problem = Problem(4,2)
Real0 = Real(0, 40); Real1 = Real(0, 40); Real2 = Real(0, 40); Real3 = Real(0, 40)

problem.types[:] = [Real0] + [Real1] + [Real2] + [Real3]
problem.function = auto_optim_v2

population_size = 20
algorithm = NSGAII(problem,population_size)
algorithm.run(10000) # Number of iterations

results1_optim_relea = np.array([algorithm.result[i].objectives[0] for i in range(population_size)])
results2_optim_relea = np.array([algorithm.result[i].objectives[1] for i in range(population_size)])

solutions_optim_relea = [[algorithm.result[i].variables[0],algorithm.result[i].variables[1],
                             algorithm.result[i].variables[2],algorithm.result[i].variables[3],
                             0,0,0,0] for i in range(population_size)]

**Pareto front and selection of the release policy**

In [14]:
fig_pf,fig_wd,fig_st,fig_pi,pareto_ens = Interactive_Pareto_front(simtime,I_for,E_for,d_for,S0,Smax,Smin,env_min,c,solutions_optim_relea,results1_optim_relea,results2_optim_relea)
widgets.VBox([widgets.HBox([widgets.VBox([fig_pi,fig_wd]),fig_pf]),widgets.HBox([fig_st])])